# Debugging markov_model

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, '../../')

import gc
import pysam
import pandas as pd
import re
import torch
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np


import helpers.train_eval as train_eval    #train and evaluation
import helpers.misc as misc                #miscellaneous functions

import encoding_utils.sequence_encoders as sequence_encoders
import encoding_utils.sequence_utils as sequence_utils
from models.spec_dss import DSSResNet, DSSResNetEmb, SpecAdd
import models.baseline.markov_model as mm
import models.baseline.markov_model_orig as mm_orig

from Bio import SeqIO
import pickle

Matplotlib is building the font cache; this may take a moment.


# Data

In [12]:
# # load the train data if it exists
# file_path = 'train_df.pickle'
# if os.path.exists(file_path):
#     with open(file_path, 'rb') as f:
#         train_df = pickle.load(f)
# else:
#     # load the fasta file and select the train data
#     fasta_file = "../../Homo_sapiens_3prime_UTR.fa"
#     sequences = []
#     for s in SeqIO.parse(fasta_file, "fasta"):
#         sequences.append(str(s.seq).upper())
#     # get the train fraction
#     val_fraction = 0.1
#     N_train = int(len(sequences)*(1-val_fraction))
#     train_data = sequences[:N_train]
#     # store it as a dataframe
#     train_df = pd.DataFrame({'3-UTR':train_data})
#     with open(file_path, 'wb') as f:
#         pickle.dump(train_df, f)
# train_df

,3-UTR
0,ATCTTATATAACTGTGAGATTAATCTCAGATAATGACACAAAATAT...
1,GGTTGCCGGGGGTAGGGGTGGGGCCACACAAATCTCCAGGAGCCAC...
2,GGCAGCCCATCTGGGGGGCCTGTAGGGGCTGCCGGGCTGGTGGCCA...
3,CCCACCTACCACCAGAGGCCTGCAGCCTCCCACATGCCTTAAGGGG...
4,TGGCCGCGGTGAGGTGGGTTCTCAGGACCACCCTCGCCAAGCTCCA...
...,...
16315,CCGTATGAAGATGTCCTGTTAAATTTACAACACTAACGATGTAGAC...
16316,ACACACCCCCGAAAAACACAAGACCGACCCAAAATCTAGAGGAAAG...
16317,AGAAGCTAAAAGGAAAGAAAATAAATCTATCAAAATTACCCTAAAC...
16318,CTTCACTTTTGGGCTCAAGGACTGTGTGAACCAACAAGGGGCCAGT...


In [2]:
# load the test data if it exists
file_path = 'test_df.pickle'
if os.path.exists(file_path):
    with open(file_path, 'rb') as f:
        test_df = pickle.load(f)
else:
    # load the fasta file and select the train data
    fasta_file = "../../Homo_sapiens_3prime_UTR.fa"
    sequences = []
    for s in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(s.seq).upper())
    # get the train fraction
    val_fraction = 0.1
    N_train = int(len(sequences)*(1-val_fraction))
    test_data = sequences[N_train:]
    # store it as a dataframe
    test_df = pd.DataFrame({'3-UTR':test_data})
    with open(file_path, 'wb') as f:
        pickle.dump(test_df, f)
test_df

,3-UTR
0,CCCCCAGAACCAGTGGGACAAACTGCCTCCTGGAGGTTTTTAGAAA...
1,TATTGAGCCCTCAGAGAGTCCACAGTCCCTCCTCTCAGTTCAGTCT...
2,TATTCATTCCAACTGCTGCCCCTCTGTCTGCCTGGCTGAGATGCAT...
3,AACGGTGCGTTTGGCCAAAAAGAATCTGCATTTAGCACAAAAAAAA...
4,TAGTTTCTAACTGTCGGACCCGTCTGTAAACCAAGGACTATGAATA...
...,...
1809,AGCAAGCATTGAAAATAATAGTTATTGCATACCAATCCTTGTTTGC...
1810,AGCAAGCATTGAAAATAATAGTTATTGCATACCAATCCTTGTTTGC...
1811,GCCTACTTCATCTCAGGACCCGCCCAAGAGTGGCCGCGGCTTTGGG...
1812,TTGTCAGTCTGTCTGCTCAGGACACAAGAACTAAGGGGCAACAAAT...


In [3]:
sequences = list(test_df["3-UTR"])
with open("../../data/exclude_motifs.pickle", "rb") as f:
    config = pickle.load(f)
print(config)


{'fixed_length': 5, 'exclude_random': [('A1CF', 'AATTA', 0), ('BOLL', 'TTTTT', 1), ('CELF1', 'TATGT', 2), ('CNOT4', 'ACACA', 3), ('DAZAP1', 'ATATA', 4), ('EIF4G2', 'GTTGC', 5), ('ESRP1', 'GGGGG', 6), ('FUBP3', 'TATAT', 7), ('HNRNPA0', 'TATAG', 8), ('HNRNPD', 'TATTA', 9), ('HNRNPDL', 'TAATT', 10), ('HNRNPK', 'GCCCA', 11), ('KHDRBS2', 'ATAAA', 12), ('KHSRP', 'TGTAT', 13), ('MBNL1', 'CGCTT', 14), ('MSI1', 'TAGTT', 15), ('NOVA1', 'TTCAT', 16), ('NUPL2', 'AAAAA', 17), ('PCBP1', 'GCCCC', 18), ('PCBP2', 'CCCCC', 19), ('PCBP4', 'ATCCC', 20), ('PRR3', 'ATAAG', 21), ('PTBP3', 'TTTCT', 22), ('RBFOX2', 'GCATG', 23), ('RBM22', 'ACCGG', 24), ('RBM24', 'GTGTG', 25), ('RBM4', 'GCGCG', 26), ('RBM41', 'TACTT', 27), ('RBM45', 'ACGCA', 28), ('RBM47', 'AATCA', 29), ('RBM6', 'CGTCC', 30), ('RC3H1', 'ATATT', 31), ('SF1', 'TAACA', 32), ('SFPQ', 'TGTAA', 33), ('SNRPA', 'TGCAC', 34), ('SRSF10', 'AGCAG', 35), ('SRSF11', 'AGGGG', 36), ('SRSF8', 'GCAGC', 37), ('SRSF9', 'AGGAG', 38), ('TARDBP', 'GTATG', 39), ('TRA2

# kmer 
* Exactly the same in both files
* Test with pseudocount = 0

In [15]:
# # training here refers to calculating the 11mer frequencies
# pseudocount = 0
# file_path = f'kmer_train_pseudocount_{pseudocount}.pickle'
# if os.path.exists(file_path):
#     with open(file_path, 'rb') as f:
#         kmer_train = pickle.load(f)
# else: 
#     # get the frequency counts of all motifs till 11mer
#     kmer_train = mm.KmerCount(11,pseudocount=pseudocount)
#     kmer_train.compute_counts(train_df['3-UTR'])
#     kmer_train.kmer_counts_dict

#     # save dictionary pickle file
#     with open('kmer_train_pseudocount_{pseudocount}.pickle', 'wb') as f:
#         pickle.dump(kmer_train, f)

In [16]:
# kmer_train.max_k

11

In [17]:
# kmer_train.kmer_counts_dict

{0: array([0.]),
 1: array([8157095., 6189764., 6274556., 8841209.]),
 2: array([2642571., 1409099., 2004685., 2087206., 2009700., 1712355.,
         338176., 2128409., 1681668., 1347883., 1649212., 1594906.,
        1818484., 1716864., 2277171., 3027915.]),
 3: array([1027296.,  390734.,  553286.,  664790.,  514728.,  347294.,
          80915.,  466033.,  587208.,  423662.,  519958.,  473737.,
         522403.,  346091.,  515909.,  702705.,  470463.,  423143.,
         614224.,  499039.,  543442.,  482336.,  103335.,  583008.,
          67580.,   81987.,   96154.,   92422.,  338932.,  496520.,
         667411.,  625424.,  551267.,  282855.,  479979.,  365720.,
         419852.,  406596.,   80568.,  440615.,  462973.,  378970.,
         456020.,  351014.,  343211.,  293355.,  487294.,  470934.,
         592164.,  311500.,  355693.,  556736.,  530657.,  474829.,
          73023.,  637847.,  562701.,  461707.,  575318.,  676946.,
         613433.,  580342.,  605556., 1228142.]),
 4: arra

# Original Markcov Model
Test bidirectional markov model with order 5 and pseudocount 0

In [9]:
# at the current state I don't have a markov matrix and can therefore not make a MarkovModel object
# therefore generate the markov matrix with BiMarkov
# bimarkov = mm_orig.BiMarkov(kmer_train)

In [18]:
# bimarkov.compile_from_counts()

/mnt/c/Users/nicol/OneDrive/Documents/TUM/ml4regmlm/project/ML4RG-2023-project/notebooks/markov_models/../../models/baseline/markov_model_orig.py:149: RuntimeWarning: invalid value encountered in divide
  self.markov_matrix[order,:,:] = self.markov_matrix[order,:,:]/np.sum(self.markov_matrix[order,:,:],axis=1)[:,np.newaxis]


In [19]:
# bimarkov.markov_matrix

array([[[0.27686247, 0.21008869, 0.21296664, 0.3000822 ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        ]],

       [[0.38741984, 0.19411721, 0.22145129, 0.19701166],
        [0.25914506, 0.23033451, 0.28098378, 0.22953665],
        [0.33129549, 0.04845012, 0.31133942, 0.30891497],
        ...,
        [       nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan]],

       [[0.48548706, 0.12552056, 0.18282317, 0.20616921],
        [0.39142342, 0.18918728, 0.20455719, 0.21483211],
        [0.38661266, 0.20732432, 0.23276384, 0.17329918],
        ...,
        [       nan,        nan,        nan,        nan],
        [       nan,        n

In [20]:
# bimarkov.serialize('markov_model.npy')

In [21]:
# del bimarkov

In [4]:
# initialize a bidirectional markov model of order 5
markov_model = mm_orig.MarkovModel(
    halflife_df = test_df,
    markov_matrix_path = 'markov_model.npy',
    order = 5,
    bidirectional = True
)

: 

: 

In [ ]:
# calculate the markov matrix using the 11mer counts
markov_model.model.compile_from_counts()

/home/lukas/Projects/ML4RG-2023-project/results/11mer/../../models/baseline/markov_model.py:154: RuntimeWarning: invalid value encountered in divide
  self.markov_matrix[order,:,:] = self.markov_matrix[order,:,:]/np.sum(self.markov_matrix[order,:,:],axis=1)[:,np.newaxis]


In [ ]:
# generate the result files needed for plotting using the test data
markov_model.test()